# Análisis SQL – Viajes de taxis en Chicago (noviembre de 2017)

Este notebook documenta una serie de consultas SQL utilizadas para analizar datos de viajes de taxis en Chicago durante noviembre de 2017.  
El análisis se centra en la actividad de las compañías de taxis, rutas relevantes y la influencia de las condiciones climáticas en la duración de los viajes.

# Análisis sintáctico de datos 

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from io import StringIO

# URL del sitio web con los datos
url = "https://practicum-content.s3.us-west-1.amazonaws.com/data-analyst-eng/moved_chicago_weather_2017.html"

# Enviar solicitud HTTP al sitio web
response = requests.get(url)

# Crear objeto BeautifulSoup para analizar el contenido HTML
soup = BeautifulSoup(response.text, "html.parser")

# Buscar la tabla con id='weather_records'
table = soup.find("table", attrs={"id": "weather_records"})

# Convertir la tabla a un string y luego usar StringIO
html_string = str(table)
weather_records = pd.read_html(StringIO(html_string))[0]

# Imprimir el DataFrame completo
print(weather_records)


## Contexto y fuente de datos

Las consultas SQL presentadas en este notebook fueron ejecutadas en el entorno SQL original proporcionado por la plataforma del curso (TripleTen / Practicum).

Debido a que no se cuenta con acceso directo a la base de datos fuera de dicho entorno, las consultas se documentan aquí con fines de transparencia y portafolio.  
Las salidas se incluyen mediante capturas de pantalla para mostrar los resultados obtenidos.


## Descripción del esquema de la base de datos

El análisis se basa en las siguientes tablas:

- **cabs**: información sobre las compañías de taxis  
- **trips**: registros individuales de los viajes  
- **neighborhoods**: identificadores y nombres de los vecindarios  
- **weather_records**: registros horarios de las condiciones climáticas  

Las consultas están escritas en sintaxis PostgreSQL.


# Esquema de la tabla

![Esquema de la tabla](Esquema_Tablas.png)

## Consulta 1 – Cantidad de viajes por compañía (15–16 de noviembre de 2017)

Esta consulta obtiene el número de viajes realizados por cada compañía de taxis entre el 15 y el 16 de noviembre de 2017.  
Los resultados se ordenan de forma descendente según la cantidad total de viajes.


In [ ]:
%%sql
SELECT
    cabs.company_name,
    COUNT(trips.trip_id) AS trips_amount
FROM
    cabs
    JOIN trips ON trips.cab_id = cabs.cab_id
WHERE
    DATE(trips.start_ts) BETWEEN '2017-11-15' AND '2017-11-16'
GROUP BY
    cabs.company_name
ORDER BY
    trips_amount DESC;


![Resultado Consulta 1](Consulta1.png)


## Consulta 2 – Viajes de compañías Yellow y Blue (1–7 de noviembre de 2017)

Esta consulta calcula el número de viajes realizados por compañías cuyo nombre contiene las palabras *Yellow* o *Blue* durante la primera semana de noviembre de 2017.


In [ ]:
%%sql
SELECT 
    cabs.company_name,
    COUNT(trips.trip_id) AS trips_amount

FROM 
    cabs
    JOIN trips ON trips.cab_id = cabs.cab_id

WHERE 
    cabs.company_name LIKE '%Yellow%'
    AND
    trips.start_ts::date BETWEEN '2017-11-01' AND '2017-11-07'

GROUP BY
    cabs.company_name

UNION

SELECT 
    cabs.company_name ,
COUNT(trips.trip_id) AS trips_amount

FROM 
    cabs
    JOIN trips ON trips.cab_id = cabs.cab_id

WHERE 
    cabs.company_name LIKE '%Blue%'
    AND
    trips.start_ts::date BETWEEN '2017-11-01' AND '2017-11-07'

GROUP BY
    cabs.company_name;

![Resultado Consulta 2](Consulta2.png)

## Consulta 3 – Compañías más populares vs. otras compañías (1–7 de noviembre de 2017)

Esta consulta compara la cantidad de viajes realizados por las dos compañías más populares (Flash Cab y Taxi Affiliation Services) frente al total de viajes realizados por el resto de las compañías.


In [ ]:
%%sql
(# Flash Cab
SELECT 
    'Flash Cab' AS company,
    COUNT(trips.trip_id) AS trips_amount
FROM 
    cabs
    JOIN trips ON trips.cab_id = cabs.cab_id
WHERE 
    cabs.company_name = 'Flash Cab'
    AND trips.start_ts::date BETWEEN '2017-11-01' AND '2017-11-07'

UNION
%%sql
# Taxi Affiliation Services
SELECT 
    'Taxi Affiliation Services' AS company,
    COUNT(trips.trip_id) AS trips_amount
FROM 
    cabs
    JOIN trips ON trips.cab_id = cabs.cab_id
WHERE 
    cabs.company_name = 'Taxi Affiliation Services'
    AND trips.start_ts::date BETWEEN '2017-11-01' AND '2017-11-07'

UNION

# Otras compañías
SELECT 
    'Other' AS company,
    COUNT(trips.trip_id) AS trips_amount
FROM 
    cabs
    JOIN trips ON trips.cab_id = cabs.cab_id
WHERE 
    cabs.company_name NOT IN ('Flash Cab', 'Taxi Affiliation Services')
    AND trips.start_ts::date BETWEEN '2017-11-01' AND '2017-11-07'

ORDER BY 
    trips_amount DESC;


![Resultado Consulta 3](Consulta3.png)

## Consulta 4 – Identificadores de vecindarios (O’Hare y Loop)

Esta consulta obtiene los identificadores de los vecindarios O’Hare y Loop, los cuales se utilizan en análisis posteriores.


In [ ]:
%%sql
SELECT 
    name, 
    neighborhood_id

FROM 
    neighborhoods

WHERE
    name IN ('O''Hare', 'Loop');

![Resultado Consulta 4](Consulta4.png)

## Consulta 5 – Clasificación de condiciones climáticas

Esta consulta clasifica las condiciones climáticas en dos categorías:
- **Malas**: lluvia o tormenta  
- **Buenas**: todas las demás condiciones


In [ ]:
%%sql
SELECT 
    ts,
    
CASE
    WHEN description LIKE '%rain%' OR description LIKE '%storm%' THEN 'Bad'
    
    ELSE 'Good'
END AS weather_conditions
    
FROM  
    weather_records;


![Resultado Consulta 5](Consulta5.png)

## Consulta 6 – Viajes del Loop a O’Hare los sábados

Esta consulta obtiene los viajes realizados los sábados desde el vecindario Loop hacia O’Hare, incluyendo:
- fecha y hora de inicio
- condiciones climáticas
- duración del viaje

Se excluyen los viajes sin información climática disponible.


In [ ]:
%%sql
SELECT
    trips.start_ts,
    CASE
        WHEN weather_records.description LIKE '%rain%' 
             OR weather_records.description LIKE '%storm%' 
        THEN 'Bad'
        ELSE 'Good'
    END AS weather_conditions,
    trips.duration_seconds
FROM
    trips
    INNER JOIN weather_records 
        ON trips.start_ts = weather_records.ts
WHERE
    trips.pickup_location_id = 50
    AND trips.dropoff_location_id = 63
    AND EXTRACT(DOW FROM trips.start_ts) = 6   -- 6 = sábado
ORDER BY
    trips.trip_id;


![Resultado Consulta 6](Consulta6.png)

Estos resultados fueron utilizados posteriormente en el análisis con Python, el cual se puede encontrar en este mismo repositorio en el archivo 